In [2]:
import numpy as np
import re
import json
import pandas as pd
import warnings
import csv
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [4]:
# Loading the data 
dataset_name = "claude9"
idx_to_label = json.load(open(f"label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
test_df = li.data_to_df(json.load(open(f"test.json", "r")))

In [5]:
test_df = test_df.sample(250, random_state=205)
eval_df = test_df.drop(columns=[ 'label', 'weak_labels'])

In [7]:
eval_df.to_csv('./claude9_eval.csv')

In [19]:
from openai import OpenAI
import os
import sys

API_KEY = ''


model_id = 'gpt-4o-2024-08-06' # replace with your model ID

os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [20]:
import openai
import pandas as pd

df = pd.read_csv('claude9_eval.csv')


categories = {
    "0": "Limitation of liability",
    "1": "Unilateral termination",
    "2": "Unilateral change",
    "3": "Content removal",
    "4": "Contract by using",
    "5": "Choice of law",
    "6": "Jurisdiction",
    "7": "Arbitration",
    "8": "Other"
}

# Function to classify text
def classify_text(text):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are an expert in legal document classification."},
            {"role": "user", "content": f"Classify the following text into one of the categories: {categories}. Text: {text}"}
        ]
    )
    classification = response.choices[0].message.content
    return classification.strip()



# Apply the function to each line of text
df['Classification'] = df['text'].apply(classify_text)

# Save the results to a new CSV file
df.to_csv('classified_file.csv', index=False)


In [23]:
df['Classification'] = df['Classification'].apply(lambda x: re.findall(r'\d+', x)[0])
df.to_csv('classified_file2.csv', index=False)


In [6]:
import pandas as pd
from sklearn.metrics import f1_score

predictions_df = pd.read_csv('classified_file2.csv')
true_labels_df = test_df

merged_df = pd.merge(predictions_df, true_labels_df, on='text')

merged_df['correct'] = merged_df['Classification'] == merged_df['label']

accuracy = merged_df['correct'].mean()
print(f"Accuracy: {accuracy:.2%}")

f1_macro = f1_score(merged_df['label'], merged_df['Classification'], average='macro')
print(f"F1 Macro Score: {f1_macro:.4f}")

print(merged_df.head())

merged_df.to_csv('comparison_with_true_labels.csv', index=False)


Accuracy: 59.60%
F1 Macro Score: 0.2948
   Unnamed: 0                                               text  \
0         901  5.6 some terms of this agreement will survive ...   
1         114  airbnb payments will process co-host services ...   
2        1290  in case of minor breaches nintendo will provid...   
3         334  airbnb may restrict the availability of the ai...   
4        1814  2.2 linden lab grants you certain licenses to ...   

   Classification  label                                        weak_labels  \
0               8      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1               8      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
2               1      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
3               3      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
4               8      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

   correct  
0     True  
1     True  
2    False  
3    False  
4     True 